##**Código documentado**

In [ ]:
#Importamos las librerías a utilizar
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

#Solicitamos permisos y montamos la carpeta de Google Drive  
drive.mount('/content/drive')

#Establecemos un máximo fijo en el ancho de las columnas
pd.options.display.max_colwidth = 150

#Leemos el archivo
key_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/M8/DataSets/datos_ejercicios/movies_etiquetado.csv')

#Pasamos las etiquetas de filas a un array en una columna
key_df=key_df.groupby(['imdb_title','imdb_id'])['tag'].apply(list).reset_index(name='etiquetas')

#Definimos una función para eliminar espacios en blanco y colocar todo en minúsculas
def limpiar(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

#Aplicamos la función limpiar a la columna "etiquetas"
key_df['etiquetas'] = key_df['etiquetas'].apply(limpiar)

#Definimos una función que convertirá a texto el contenido de la columna "etiquetas"
def juntar(x):
    return ' '.join(x['etiquetas'])

#Aplicamos la función a todas filas
key_df['etiquetas'] = key_df.apply(juntar, axis=1)

#Eliminamos las palabras comunes (stopwords) en la columna "etiquetas", especificamos el idioma
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(key_df['etiquetas'])

#Calculamos la similitud coseno
similitud_coseno = cosine_similarity(count_matrix, count_matrix)

#Utilizamos el índice del título y el título de la película para crear una Serie para poder utilizarla en la función de recomendación
indices = key_df.reset_index(drop=True)
indices = pd.Series(indices.index, index=key_df['imdb_title'])

#Definimos la función de recomendación
def content_recommender(title, cosine_sim=similitud_coseno, df=key_df, indices=indices):
    #Obtenemos el índice
    id_ = indices[title]

    #Filtramos por el índice y devolvemos una tupla con el index(enumerate)y el resultado de la similitud coseno
    scores = list(enumerate(cosine_sim[id_]))

    #Ordenamos en base a la similitud coseno
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    #Nos quedamos con los 6 primeros
    scores = scores[1:6]

    #Extraemos los índices
    indices = [i[0] for i in scores]

    #Devolvemos las 6 más similares
    return df['imdb_title'].iloc[indices]





In [6]:
#Llamamos la función con la película "Aladdin" para ver los resultados 
content_recommender('Aladdin', similitud_coseno, key_df, indices)


528       Train of Life
430    The Indian Scarf
316    Restless Natives
356          Stir Crazy
277    Of Human Bondage
Name: imdb_title, dtype: object

In [34]:
#Definimos una función de recomendación de películas aleatorias 
def content_recommender_random (df = key_df):
    title = df['imdb_title'].iloc[random.randint(0,len(key_df))]
    print ("Estas son las recomendaciones para " + title)
    return (content_recommender(title, similitud_coseno,key_df,indices))

#Llamamos a la función de películas aleatorias 
content_recommender_random()

Estas son las recomendaciones para The End of Summer


469          The Pawnshop
528         Train of Life
111    Death of a Cyclist
383     The Bohemian Girl
356            Stir Crazy
Name: imdb_title, dtype: object